In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy.optimize import minimize
from scipy.optimize import least_squares


In [2]:
from numpy.random import default_rng, SeedSequence

sq = SeedSequence()
seed = sq.entropy        # on sauve la graine pour reproduire les résultats
print('seed = ', seed)
rng = default_rng(sq)
rng.standard_normal(5)

seed =  261872297657270704497041869730793872727


array([ 0.13526605, -0.6145526 ,  0.22542003, -0.92730968, -1.14693194])

In [3]:
sns.set_style("whitegrid")
mpl.rcParams['figure.dpi'] = 100

params_grid = {"color": 'lightgrey', "linestyle": 'dotted', "linewidth": 0.7 }

In [4]:
# Pamètre du model
N = 10
M = 10000

T = 1
dt = T/N
S0 = 100
K = 100
sigma = 0.1
r = 0.04
m = 3

We are going to compare the result of the LS algorithm when using a linear regression and when using a Neural Network (Non linear regression) to learn the continuation function.

We are going to code the Longstaff-Schwartz algorithm for an American Put with respect to a Black Scholes model, that is to say a put that we can exerce at $t_k = \frac{k}{N}$ where $k = 0,..., N$. We will take $r=0.04$, $\sigma=0.1$, $x_0=100$ and the strike $K=100$ with $N=10$ dates until $T=1$ and the payoff $\phi_k(x) = e^{-r k/N}(K-x)_+$.

For that we are going to reuse the notation of the 2nd project :

In [5]:
def brownian_1d(n_times: int, n_paths: int,
                final_time: float=1.0,
                increments: bool=False,
                random_state: np.random.Generator=rng) -> np.array:
    """Simulate paths of standard Brownian motion
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths
        final_time: Final time of simulation
        increments: If `True` the increments of the paths are returned.
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths if the argument `increments` is `False`
        `np.array` of shape `(n_times, n_paths)` containing the increments if the argument `increments` is `True`
    """
    dB = np.sqrt(final_time / n_times) * random_state.standard_normal((n_times, n_paths))
    if increments:
        return dB
    else:
        brownian = np.zeros((n_times+1, n_paths))
        brownian[1:] = np.cumsum(dB, axis=0)
        return brownian

def black_scholes_1d(n_times: int, n_paths: int,
                     final_time: float=1.0,
                     random_state: np.random.Generator=rng, *,
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    """Simulate paths of Black-Scholes process
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths
        final_time: Final time of simulation
        init_value: `S0`
        r: Interest rate
        sigma: Volatility
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths
    """
    Bt = brownian_1d(n_times, n_paths)
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

To find the value $V_0$, we apply the following backward recurrence :
$V_N(x) = \phi_N(x)$

and $V_n(x) = max(\phi_n(x), \mathbb{E}[V_{n+1}(x))$

The aim of the LS algorithm is to approch the conditional expectation with function.

For that we use the following property :
$$\theta^* = \arg \min_{\theta} \mathbb{E}[(\mathbb{E}(Y|X) - \Phi(X, \theta))^2]$$
$$ = \arg \min_{\theta} \mathbb{E}[(Y - \Phi(X, \theta))^2]$$
In the following example we will use Linear Regression, the algorithm become :
$$V_N(x) = \phi_N(x)$$

$$V_n(x) = max(\phi_n(x), \Phi(x,\theta_n))$$

with $\theta_n = argmin_\theta \sum_{j=1}^{M} \left( V_{n+1}(X_{n+1}^{(j)}) - \Phi(X_{n}^{(j)}, \theta) \right)^2$

For $\Phi$ we can choose between two families of function :

$\Phi(x) = \theta_0+\theta_1*x+\theta_2*x^2$

or

 $\Phi(x) = \theta_0+\theta_1*x+\theta_2*x^2+\theta_3*max(K-x, 0)$

 In the following, I used the first version $\Phi$ but we will found the result for the first version.

In [6]:
# psi est ma régression linéaire qui va me permettre d'approcher le résultat théorique
# Le but étant de trouver les theta optimaux
def psi(x, theta, m):
    res = 0
    for i in range(m):
      res += theta[i]*(x**i)
    return res

def phi(x, K, k, N):
    return np.exp(-r*k/N)*np.maximum(K-x, 0)

#same function as the previous one but it allows to manipulate matrix
#it will be useful when I will use a regression to find theta based on the prices obtained by Black Scholes simulation
def phi_regression(x, K, k, N):
    return np.exp(-r*k/N)*np.maximum(K-x[k], 0)

#Function where we want to find the argmin theta
def objective_function(theta, M, S, n, K, thetas):
    result = 0
    for j in range(M):
        result += (V(S[n+1][j], n+1, thetas[-1], N, K) - psi(S[n][j], theta, K))**2
    return result/M

def V(x, n, theta,N, K):
    if n == N:
        return phi(x,K,N,N)
    else:
        return max(phi(x, K,n,N), psi(x, theta, K))

def simulate_V(S,K,r,n,V):
    if n==-1:
        return V
    else :
        V[n] = np.maximum(phi_regression(S,K,n,N),V[n+1])
        return simulate_V(S,K,r,n-1,V)


# Régression Linéaire



Let $\left( e_{k}(x) \right)_{k \leq L}$ a function family $e_{k} : \mathbb{R}^{d} \rightarrow \mathbb{R}$.

We assume $\forall n, \, n = 1, \dots, N-1 \times (e_{k}(X_n))_{k \leq L}$, is a total generative family of $L^{2}(\sigma(X_n))$

Let $m$ finite, (linearly independant family, ie $(e_k(X_n))_{1 \leq k \leq m}$ is a base).

We take the orthogonal projection on  $\text{Vect}(e_1(X_n), \dots, e_m(X_n))$, ie :


$$\Phi_{m}(x ; \theta) = \sum_{k=1}^{m} \theta_k e_k(x), \quad \theta = (\theta_1, \dots, \theta_m)^T$$

The optimal stopping time is given by :

$$τ_N^m = M$$
$$τ_n^m = n1_{ϕ_{n}(X_n)>\Phi_{m}(x ; \theta_n^m)} + τ_{n+1}^m1_{ϕ_{n}(X_n)<\Phi_{m}(x ; \theta_n^m)}$$


**Important** : At each step $n$, we need to determine $\theta$ such that :

For simplicity we denote : $Z_{n+1}^{(m)} = Z_{τ_{n+1}^m}$

$$\arg \min_{\theta \in \mathbb{R}^m} E \left[ \left( Z_n^{(m)} - \sum_{k=1}^{m} \theta_k e_k(X_n) \right)^2 \right]$$

We minimize thee funciton $W(\theta)$ defined by :


$$W(\theta) = E \left[ \left( Z_n^{(m)} - \sum_{k=1}^{m} \theta_k e_k(X_n) \right)^2 \right]$$

To find a minimum of W  $W: \mathbb{R}^m \rightarrow \mathbb{R}$, we can search a point $\theta^*$ that cancels the gradient.

The gradient of $W$ is given by :


$$\nabla W(\theta) : \mathbb{R}^m \rightarrow \mathbb{R}^m$$


$$\nabla W(\theta) = E \left[ 2 \left( Z_n^{(m)} - \sum_{k=1}^{m} \theta_k e_k(X_n) \right) e_j(X_n) \right]$$

Now, we introduce the following notation :


$$\theta = (\theta_1^{(m)}, \dots, \theta_m^{(m)})^T$$


$$Z_n^{(m)} = \sum_{k=1}^{m} \theta_k^{(m)} e_k(X_n), \quad \text{et} \quad e^{(m)}(X_n) = (e_1(X_n), \dots, e_m(X_n))^T$$

We then have :


$$\nabla W(\theta) = -E \left[ \left( Z_n^{(m)}(x) - e^{(m)}(X_n)^T \theta \right) e^{(m)}(X_n) \right]$$

The gradient is null in one point $\theta = \left[ A_n^{(m)} \right]^{-2} E \left[ Z_n^{(m)}(X) e^{(m)}(X_n) \right]$ with :


$$A_n^{(m)} = e^{(m)}(X_n) e^{(m)}(X_n)^T$$

(If the matrix $A_n^{(m)}$ is inversible)

In practice, we use emprical laws from a sample$\left( X_n^{(i)} \right)_{i = 0, \dots, N}$ :


$$\hat{\theta}_n^{(m, H)} = \left[ \hat{A}_n^{(m, H)} \right]^{-1} \sum_{i=1}^{M} Z_n^{(i)} e^{(m)}(X_n^{(i)})^T$$

With :


$$\hat{A}_n^{(m, H)} = \frac{1}{M} \sum_{i=1}^{M} e^{(m)}(X_n^{(i)}) e^{(m)}(X_n^{(i)})^T$$



In [7]:
def regression_theta(X_t, M, m, n):
    e_m_X = []
    for j in range(M):
        e_m_X_j = [X_t[n][j]**k for k in range(m)]

        e_m_X.append(np.array(e_m_X_j).reshape(-1, 1))

    A_n_m_M = np.zeros((e_m_X[0].dot(e_m_X[0].T).shape))
    for i in range(M):
        A_n_m_M += e_m_X[j].dot(e_m_X[j].T)
    A_n_m_M = (1/M) * A_n_m_M

    e_m_X = np.array(e_m_X)
    V_simu = np.zeros_like(X_t)
    V_simu[-1] = phi_regression(X_t, K, N, N)
    Z_n_m =  np.array([simulate_V(X_t[:,j],K,r,V_simu.shape[0]-2,V_simu) for j in range(M)])

    esperance = [0 for _ in range(m)]
    for j in range(M-1):
        for k in range(m):
            esperance[k] += Z_n_m[n+1][Z_n_m.shape[1]-1][j]* e_m_X[j][k][0]
    esperance = [esperance[k] / M for k in range(m)]

    theta_n_m_M = np.linalg.inv(A_n_m_M+0.00001*np.identity(len(A_n_m_M[0]))).dot(esperance)
    return theta_n_m_M

def longstaff_Schwartz_regression(x0, K, T, N, M, m):
    V_hat = [phi(x0, K,N, N)]
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = []
    for n in reversed(range(N)):
        print(n)
        theta = regression_theta(S_t, M, m, n)
        optimized_theta = theta#.x
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi(x0, optimized_theta, m)))
        thetas.append(optimized_theta)
    return V_hat, thetas, S_t

In [8]:
V_hat, thetas, S_t = longstaff_Schwartz_regression(S0, K, T, N, M, 3)
print(V_hat)

9
[  99359.80021206 5140868.31868033  -41996.63850465]
8
[  80214.73892131 4063619.58484872  -35275.29689062]
7
[  85327.46326978 4412846.82004853  -37174.64190373]
6
[  84902.76459599 4435150.40718442  -37033.06587941]
5
[  62813.74232874 3197093.740574    -28476.20416424]
4
[  72360.05820395 3786376.30098276  -32347.60880932]
3
[  52298.62436892 2681423.40069135  -24157.44423601]
2
[  34967.87012467 1770237.15756574  -16571.71037624]
1
[  20214.85678819 1014381.40342797   -9771.34587879]
0
[1.83346099e-07 2.15059961e-06 2.21370562e-04]
[0.0, 94219806.62173903, 53689204.31758779, 69623590.43078882, 73269284.68896538, 35010146.157340884, 55233902.063266814, 26620196.333433777, 11341579.864268005, 3744896.41168727, 2.213920860723911]


In [33]:
print("The price of an American Put with linear Regression is", V_hat[-1], "$")

The price of an American Put with linear Regression is 2.213920860723911 $


We obtained a result closed to the theorical price (2.6$). This is certainly du to the fact that my regression is an approximatioin of theorical formulas but also to the fact that we approach the expectation by using MC.

## Réseaux de neurones

In [10]:
!pip install torch torchvision


# Approximation by a MLP (Multilayer Perceptron)
FNN (Feedforward Neural Networks Fully Connected)

Let $\Phi: \mathbb{R}^d \rightarrow \mathbb{R}$ the function we search to approximate the conditional expectation of the LS algorithm


$$x \mapsto \Phi(x ; \Theta)$$

This is seen as a composition of functions for each layer $L \geq 2$:


$$\Phi = A_L \circ \sigma \circ A_{L-1} \circ \sigma \circ \dots \circ A_1$$

where, for $l = 1, \dots, L$:


$$A_l : \mathbb{R}^{d_{l+1}} \rightarrow \mathbb{R}^{d_l}, \quad \text{a linear function} \quad A_l(x) = W_l x + \beta_l$$

- $W_l \in \mathbb{R}^{d_{l+1}, d_l}$
- $\beta_l \in \mathbb{R}^{d_l}$

**Note:**

- $d_1 = d$ is the dimension of the inputs.
- Here $d_L = 1$.
- All parameters $(W_l, \beta_l)$ for $l = 1, \dots, L$ are grouped in the notation $\Theta$.

The function $\sigma$ is called the activation function $\sigma: \mathbb{R} \rightarrow \mathbb{R}$ applied component-wise to its argument. A typical choice is $\sigma(x) = \text{ReLU}(x)$.

### Number of parameters:
The number of parameters is given by:


$$\sum_{l=1}^{L} (d_l + 1) \times d_{l+1}$$

In the following, we consider functions with intermediate dimensions fixed to $m$, i.e., $d_1 = d_2 = \dots = d_{L-1} = m$.

We note:


$$\mathcal{N}_m = \left\{ \hat{I}_m(x; \theta): \mathbb{R}^d \rightarrow \mathbb{R}, \, \theta \in \mathbb{R}^p \right\}$$


$$\mathcal{N}_m = \left\{ \hat{I}_m : \mathbb{R}^d \rightarrow \mathbb{R}, \, \theta \in \mathbb{R}^p \right\} \quad \text{with} \quad p = m (d + 1) + m^2(L - 2)$$

We also consider:


$$\mathcal{N}_{\infty} = \bigcup_m \mathcal{N}_m$$

---

## Theorem: Universal Approximation in $L^2$

Let $\sigma$ be continuous and bounded (but not a ReLU function).  
Let $\mu$ be a probability measure on $\mathbb{R}^d$.  
Then, for all $L \geq 2$, $\mathcal{N}_{\infty}$ is dense in $L^2(\mathbb{R}^d, \mu)$.

---

### Result (Hornik, 1991)

### Application:
Let $E[Z | X]$ be a random variable in $L^2(\mathbb{R}^d, \mu)$.  
Then, we can approximate this random variable by an element of $\mathcal{N}_m$ such that there is a sequence of elements of $\mathcal{N}_m$ such that:


$$\lim_{m \rightarrow \infty} E\left[ \left( E[Z | X] - \Phi_m(X; \Theta_m) \right)^2 \right] = 0$$

---

### How to solve?

We need to minimize $\Theta^{(m)}$ such that:


$$\Theta^{(m)} = \arg \min_{\Theta \in \mathbb{R}^p} E\left[ \left( Z_n^{(m)} - \Phi_m(X_n; \Theta) \right)^2 \right]$$

As before (linear case), we look for a $\Theta$ that cancels the gradient:


$$\nabla W(\Theta) = -E\left[ 2 \nabla \Phi_m(X_n, \Theta) \left( z_n^{(m)} - \hat{I}_m(X_n; \Theta) \right) \right]$$

If the function can be differentiated, proceed using a gradient descent method.


In [23]:
## Definition of the previous function by using Pytorch
def V_pytorch(x, n, models, N, K):
    if n==N:
        return torch.tensor(phi(x,K,n,N))
    else :

        return torch.tensor(np.maximum(phi(x,K,n,N),models[n].forward(torch.tensor(x, dtype=torch.float32).unsqueeze(1))))

In [24]:
import torch
from torch import nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

m = 4
class NeuralNetwork(nn.Module):
    def __init__(self,m):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, m),
            nn.ReLU(),
            nn.Linear(m, m),
            nn.ReLU(),
            nn.Linear(m, m),
            nn.ReLU(),
            nn.Linear(m, 1)
        )



    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out

In [27]:
# Here we train our Neural Network bu using the loss :
def nn_loss(model, S_t, K, n, N, models):
    yy = V_pytorch(S_t[n+1], n+1, models, N, K)
    S_n = torch.tensor(S_t[n], dtype=torch.float32).unsqueeze(1)
    xx = model.forward(S_n)
    loss = ((yy - xx) ** 2).mean()
    return loss

M_nn=5000

def longstaff_Schwartz_nn(x0, K, T, N, M, m, num_epochs):

    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=S0, r = r, sigma = sigma)
    models = np.array([NeuralNetwork(m) for i in range(N)])
    for i in reversed(range(N)):
        model = NeuralNetwork(m)
        optimizer = optim.SGD(model.parameters(), lr=0.01)
        losses = []
        for epoch in range(num_epochs):
            loss = nn_loss(model, S_t, K, i, N, models)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            print(epoch)
            print(loss)
        for param in model.parameters():
            param.requires_grad_(False)
        models[i] = model
    price = max(phi(S0, K,0,N), models[0].forward(torch.tensor(S0, dtype=torch.float32).unsqueeze(-1)))
    return price

In [30]:
price = longstaff_Schwartz_nn(S0, K, T, N, M_nn, 4, 20)
print(price)

0
tensor(20.8576, dtype=torch.float64, grad_fn=<MeanBackward0>)
1
tensor(20.6997, dtype=torch.float64, grad_fn=<MeanBackward0>)
2
tensor(20.5493, dtype=torch.float64, grad_fn=<MeanBackward0>)
3
tensor(20.4059, dtype=torch.float64, grad_fn=<MeanBackward0>)
4
tensor(20.2691, dtype=torch.float64, grad_fn=<MeanBackward0>)
5
tensor(20.1385, dtype=torch.float64, grad_fn=<MeanBackward0>)
6
tensor(20.0138, dtype=torch.float64, grad_fn=<MeanBackward0>)
7
tensor(19.8947, dtype=torch.float64, grad_fn=<MeanBackward0>)
8
tensor(19.7808, dtype=torch.float64, grad_fn=<MeanBackward0>)
9
tensor(19.6718, dtype=torch.float64, grad_fn=<MeanBackward0>)
10
tensor(19.5676, dtype=torch.float64, grad_fn=<MeanBackward0>)
11
tensor(19.4678, dtype=torch.float64, grad_fn=<MeanBackward0>)
12
tensor(19.3723, dtype=torch.float64, grad_fn=<MeanBackward0>)
13
tensor(19.2808, dtype=torch.float64, grad_fn=<MeanBackward0>)
14
tensor(19.1932, dtype=torch.float64, grad_fn=<MeanBackward0>)
15
tensor(19.1093, dtype=torch.floa

<ipython-input-23-eb8e585c9f6e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(np.maximum(phi(x,K,n,N),models[n].forward(torch.tensor(x, dtype=torch.float32).unsqueeze(1))))


0
tensor(22.8172, dtype=torch.float64, grad_fn=<MeanBackward0>)
1
tensor(63879.3607, dtype=torch.float64, grad_fn=<MeanBackward0>)
2
tensor(56.4432, dtype=torch.float64, grad_fn=<MeanBackward0>)
3
tensor(46.2595, dtype=torch.float64, grad_fn=<MeanBackward0>)
4
tensor(37.2283, dtype=torch.float64, grad_fn=<MeanBackward0>)
5
tensor(29.3042, dtype=torch.float64, grad_fn=<MeanBackward0>)
6
tensor(22.8185, dtype=torch.float64, grad_fn=<MeanBackward0>)
7
tensor(18.1524, dtype=torch.float64, grad_fn=<MeanBackward0>)
8
tensor(15.3693, dtype=torch.float64, grad_fn=<MeanBackward0>)
9
tensor(14.0523, dtype=torch.float64, grad_fn=<MeanBackward0>)
10
tensor(13.5628, dtype=torch.float64, grad_fn=<MeanBackward0>)
11
tensor(13.4151, dtype=torch.float64, grad_fn=<MeanBackward0>)
12
tensor(13.3768, dtype=torch.float64, grad_fn=<MeanBackward0>)
13
tensor(13.3677, dtype=torch.float64, grad_fn=<MeanBackward0>)
14
tensor(13.3657, dtype=torch.float64, grad_fn=<MeanBackward0>)
15
tensor(13.3652, dtype=torch.f

In [32]:
print("The price of an American Put with Neural Network", price[0].item(), "$")

The price of an American Put with Neural Network 2.0320379734039307 $


We observe that with only 5000 trajectories and 20 epochs we are closed to the theorical price. However this method is much more longer than the one using Linear Regression